In [ ]:
import requests
import json
from pandas import Series, DataFrame
from google.colab import files


request1 = requests.get('http://api6.card-gorilla.com:8080/v1/charts/ranking?date=2020-11-03&term=weekly&card_gb=CRD&limit=100&chart=top100&idx=&idx2=') #json 불러오기
html1 = request1.text #json파일을 text로 저장
cardssite = json.loads(html1) #cardssite 리스트에 json파일을 저장/cardsite는 list

totalbenefit = [] #카드혜택목록들 저장할 리스트
cardsinfo = {} #각 카드마다의 정보를 사전형식으로 저장

for eachcard in cardssite: 
    request2 = requests.get('http://api6.card-gorilla.com:8080/v1/cards/' + str(eachcard['card_idx'])) #각각 카드들 사이트를 접속, 정보 불러옴
    html2 = request2.text
    eachcardinfo = json.loads(html2)
    cardsinfo[eachcard['card_idx']] = eachcardinfo #각 카드마다의 정보를 카드고유번호에 맞게 저장
    for benefit in eachcardinfo['key_benefit']: #카드정보들 중 혜택들
        if benefit['title'] != '유의사항': #필요없는 값 제외
            totalbenefit.append(benefit['title']) 

benefit_set = set(totalbenefit) #중복 제외

totalbenefit = sorted(list(benefit_set)) #가나다순 정리

cardsdic = {} #추출한 카드별 혜택 사전식으로 저장

for eachcard in cardssite: 
    cardsdic[eachcard['card_idx']] = {} #key로에카드고유번호 
    cardsdic[eachcard['card_idx']]['name'] = eachcard['name'] #사전 안에 사전, name이라는 key에 각자의 카드 이름 value로 저장
    for benefit in totalbenefit: #혜택들(benefit) 한개씩 불러옴
        cardsdic[eachcard['card_idx']][benefit] = '' #각자의 카드에 카드 혜택들 key 생성 value는 ''으로 저장
    cardinformation = cardsinfo[eachcard['card_idx']] #각 카드 고유번호로 카드정보들이 저장된 사전에서 그 카드의 정보 가져옴
    for benefit in cardinformation['key_benefit']: #가져온 정보 중 key_benefit으로 주요혜택만 추출
        if benefit['title'] != '유의사항': #필요없는 값 제외
            cardsdic[eachcard['card_idx']][benefit['title']] = benefit['comment'] #혜택이 존재한다면 혜택내용 저장

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 카드별 혜택 구분

df = {'name': []} #데이터프레임으로 만들 사전

for benefit in totalbenefit:
    df[benefit] = [] #각 혜택들은 일단 빈 배열

for key1, value1 in cardsdic.items(): #카드고유번호에 따른 정보
    for key2, value2 in value1.items(): #카드의 정보에서 각각의 혜택 빼옴
        df[key2].append(value2) #key2 = name,title , value2 = name,comment

df = DataFrame(df)
df.to_csv('Top100_card_key_benefits.csv') 
files.download('Top100_card_key_benefits.csv')

In [ ]:
#카드 구분 없이 상위100개 카드 혜택들 종합

text = ''
for key1,value1 in cardsdic.items():
  for key2,value2 in value1.items():
    if key2 != 'name' and value2 != '':
      text = text + value2 + '\n'

text = text.replace('\r','')

print(text)

with open('card_benefits_text.txt', 'w') as f:
    for item in text:
        f.write("%s" % item)
f.close()